 # **RFM Analysis**

This data analysis study attempts to provide insight into a superstore's consumer behavioural segmentation over a one-year period.  The objective are to obtain a better understanding of customer purchase activities and identify the customer segments that should be targeted or avoided. Further analysis is carried out to determine the customer behaviour  in relation to the discount.

The analysis use The RFM analysis method. RFM is based on Recency, Frequency, and Monetary value of customer's purchases.
- recency - how recently a customer has purchased,
- frequency - how often they purchase,
- monetary - how much the customer spends


In [3]:
# load library
import pandas as pd
from datetime import datetime


### 1. **Data Preparation**

In [4]:
# load the dataset
file = 'superstore_dataset2011-2015.csv'
data = pd.read_csv(file, encoding='cp1252')
# get sumary of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

In [5]:
# clean column names
data.columns = [col.lower().replace(' ', '_') for col in data.columns]
# select data to work with
df = data[['order_id', 'order_date', 'customer_id', 'sales', 'quantity', 'discount']]

# create a copy before modifying dataframe
df = df.copy()

# convert order_date column dtype from objecy to dataframe
df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=False, format='mixed')

# show the summary of working dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     51290 non-null  object        
 1   order_date   51290 non-null  datetime64[ns]
 2   customer_id  51290 non-null  object        
 3   sales        51290 non-null  float64       
 4   quantity     51290 non-null  int64         
 5   discount     51290 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 2.3+ MB


In [6]:
# get the year of order date as a new column
df['year'] = df['order_date'].dt.year

# check size of data each year
df['year'].value_counts().reset_index()

,year,count
0,2014,17531
1,2013,13799
2,2012,10962
3,2011,8998


In [7]:
# set working dataframe (wd) and select 2014 data only
wd = df.loc[df.year == 2014]

### 2. Customer Segmentation

In [8]:
# calculate RFM Values

# set dictionary for data aggregation
agg_dict = {'order_id': 'count', 'sales': 'sum', 'order_date': 'max'}

# calculate frequency, monetary and max_date values
wd_rfm = wd.groupby('customer_id').agg(agg_dict).reset_index()

# rename the columns to the correct naming
wd_rfm.columns = ['customer_id', 'frequency', 'monetary', 'max_date']

# set analysis date
working_date = datetime.strptime('2015-01-01', '%Y-%m-%d')

# calculate recency values
wd_rfm['recency'] = wd_rfm['max_date'].apply(lambda x: (working_date-x).days)

# drop 'max_date' column
wd_rfm.drop(['max_date'], axis=1, inplace=True)

# arangge the columns in RFM order
wd_rfm = wd_rfm.reindex(columns=['customer_id', 'recency', 'frequency', 'monetary'])

# show the result tables
wd_rfm.tail()

,customer_id,recency,frequency,monetary
1506,YS-21880,10,19,7282.4740
1507,ZC-11910,201,1,7.1730
1508,ZC-21910,4,27,4922.8390
1509,ZD-11925,4,8,856.2600
1510,ZD-21925,2,6,2029.9389


In [9]:
# determine the RFM scores

r_labels, f_labels, m_labels = range(4, 0, -1), range(1,5), range(1,5)

# segment rfm values into 4 groups
wd_rfm['r_score'] = pd.qcut(wd_rfm['recency'], q=4, labels=r_labels).astype(int)
wd_rfm['f_score'] = pd.qcut(wd_rfm['frequency'], q=4, labels=f_labels).astype(int)
wd_rfm['m_score'] = pd.qcut(wd_rfm['monetary'], q=4,labels=m_labels ).astype(int)

# create the score: method no. 2 - make a code using 
wd_rfm['rfm_code'] = wd_rfm['r_score'].astype(str) + wd_rfm['f_score'].astype(str) + wd_rfm['m_score'].astype(str)

# create the score: method no 1 - sum the RFM scores
wd_rfm['rfm_sum'] = wd_rfm['r_score'] + wd_rfm['f_score'] + wd_rfm['m_score']

# show the result tables
wd_rfm.tail()

,customer_id,recency,frequency,monetary,r_score,f_score,m_score,rfm_code,rfm_sum
1506,YS-21880,10,19,7282.4740,4,4,4,444,12
1507,ZC-11910,201,1,7.1730,1,1,1,111,3
1508,ZC-21910,4,27,4922.8390,4,4,4,444,12
1509,ZD-11925,4,8,856.2600,4,2,2,422,8
1510,ZD-21925,2,6,2029.9389,4,2,3,423,9


In [10]:
# calculate average discount of each customers
wd_disc = wd.groupby('customer_id').agg({'discount': 'mean'}).reset_index()

# join the wd_rfm and wd_disc dataframe
wd_rfm_disc = pd.merge(wd_rfm, wd_disc, on='customer_id')

# show the summary of working dataset
wd_rfm_disc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  1511 non-null   object 
 1   recency      1511 non-null   int64  
 2   frequency    1511 non-null   int64  
 3   monetary     1511 non-null   float64
 4   r_score      1511 non-null   int32  
 5   f_score      1511 non-null   int32  
 6   m_score      1511 non-null   int32  
 7   rfm_code     1511 non-null   object 
 8   rfm_sum      1511 non-null   int32  
 9   discount     1511 non-null   float64
dtypes: float64(2), int32(4), int64(2), object(2)
memory usage: 94.6+ KB


In [11]:
wd_rfm.head()

,customer_id,recency,frequency,monetary,r_score,f_score,m_score,rfm_code,rfm_sum
0,AA-10315,9,17,3889.2065,4,3,3,433,10
1,AA-10375,7,14,1904.5380,4,3,2,432,9
2,AA-10480,118,10,7752.9070,1,2,4,124,7
3,AA-10645,27,19,3539.8788,3,4,3,343,10
4,AA-315,3,3,787.3920,4,1,2,412,7


In [12]:
# determine customer segments

loyalist_code = ['344', '434', '443']
potential_code = ['333', '334', '343', '433']

def assign_label (wd_rfm):
  if wd_rfm['rfm_code'] == '444':
    return 'champion'
  if wd_rfm['rfm_code'] in loyalist_code:
    return 'loyalist'
  if wd_rfm['rfm_code'] in potential_code:
    return 'potential_loyalist'
  else:
    return 'other'

wd_rfm['label'] = wd_rfm.apply(assign_label, axis=1)

In [13]:
wd_rfm.head()

,customer_id,recency,frequency,monetary,r_score,f_score,m_score,rfm_code,rfm_sum,label
0,AA-10315,9,17,3889.2065,4,3,3,433,10,potential_loyalist
1,AA-10375,7,14,1904.5380,4,3,2,432,9,other
2,AA-10480,118,10,7752.9070,1,2,4,124,7,other
3,AA-10645,27,19,3539.8788,3,4,3,343,10,potential_loyalist
4,AA-315,3,3,787.3920,4,1,2,412,7,other


### 3. Data Analysis

### 3.1 Descriptive analytics

Descriptive statistics are useful to describe and present a series of observations in a concise and informative way. There are two families of descriptive statistics: location and dispersion measures. Location measures give information about the position of the data, whereas dispersion measures give information about the variability of the data [1].

we compute the location  measures (minimum, maximum, mean, median) and dispersion measures (standard deviation, coeficient of variance) using pandas function pd.describe() 


In [14]:
# get a descriptive statistic measure using pd.describe() function
wd_rfm_descriptive = wd_rfm[['recency', 'frequency', 'monetary']].describe().round(1)
# calculate coeficient of variance with the formula: standard deviation divided by mean
wd_rfm_descriptive.loc['co_var'] = (wd_rfm_descriptive.loc['std']/wd_rfm_descriptive.loc['mean']).apply(lambda x: f"{x:.0%}")
#show the result data
wd_rfm_descriptive


,recency,frequency,monetary
count,1511.0,1511.0,1511.0
mean,66.0,11.6,2845.7
std,74.7,8.5,2906.5
min,1.0,1.0,2.1
25%,14.0,4.0,530.0
50%,37.0,10.0,1919.8
75%,93.0,17.0,4372.4
max,363.0,48.0,23295.2
co_var,113%,73%,102%


all the median of the recency, frequency and monetary data are less than the mean indicate the data is righ-skewed or long right tail distribution. meaning the more customers have average to low recency, frequency, monetary score.

In [15]:
# descriptive statistic of the recency score 
rec_stat = wd_rfm.groupby('r_score').agg({'recency': ['count','min', 'max']}).reset_index()
rec_stat.columns = ['r_score', 'r_count', 'r_min', 'r_max']

# descriptive statistic of the frequency score 
freq_stat = wd_rfm.groupby('f_score').agg({'frequency': ['count','min', 'max']}).reset_index()
freq_stat.columns = ['f_score', 'f_count', 'f_min', 'f_max']

# descriptive statistic of the monetary score 
mon_stat = wd_rfm.groupby('m_score').agg({'monetary': ['count','min', 'max']}).reset_index().round()
mon_stat.columns = ['m_score', 'm_count', 'm_min', 'm_max']

# merge as one dataframe
all_rfm_stat = pd.concat([rec_stat, freq_stat], axis=1)
all_rfm_stat= pd.concat([all_rfm_stat, mon_stat], axis=1)
all_rfm_stat

,r_score,r_count,r_min,r_max,f_score,f_count,f_min,f_max,m_score,m_count,m_min,m_max
0,1,375,94,363,1,392,1,4,1,378,2.0,529.0
1,2,377,38,93,2,405,5,10,2,378,531.0,1920.0
2,3,368,15,37,3,341,11,17,3,377,1920.0,4372.0
3,4,391,1,14,4,373,18,48,4,378,4373.0,23295.0


In [16]:
wd_rfm['label'].value_counts()

label
other                 1057
potential_loyalist     174
loyalist               163
champion               117
Name: count, dtype: int64